# Task 001

In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import tool, Tool, load_tools, initialize_agent, AgentType
from langchain.chains.conversation.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import ConversationalChatAgent
from langchain.chains import SimpleSequentialChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
# import langchain
## langchain.debug = True
# langchain.debug = False
# from langchain.agents.format_scratchpad import format_to_openai_functions
# from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools import DuckDuckGoSearchRun
# from langchain.tools.render import format_tool_to_openai_function
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [3]:
teammates = {
    "Ben (CEO)": {
        "personality": "Friendly and supportive",
        "knowledge_base": "Experienced in business strategy",
    },
    "Tyne (CTO)": {
        "personality": "Analytical and technical",
        "knowledge_base": "Expert in machine learning and data pipelines",
    },
    "Da (Assistant)": {
        "personality": "Helpful and informative",
        "knowledge_base": "Access to a wide range of information",
    },
}

incidents = [{
    "description": "Data pipeline issue causing recommendation model degradation",
    "impact": "Revenue loss and customer dissatisfaction",
    "assigned_to": "Tyne (CTO)",
    },
    {
    "description": "Issue with recent commit to the production environment",
    "impact": "Revenue loss and customer dissatisfaction",
    "assigned_to": "Tyne (CTO)",
    },
    {
    "description": "The way data is stored",
    "impact": "Revenue loss and customer dissatisfaction",
    "assigned_to": "Tyne (CTO)",
    }
]

## Duck Duck go search

In [4]:
ddg_search = DuckDuckGoSearchRun()

ddg_search_tool = Tool(
    name="Duck Duck go Search",
    description="Search Duck Duck go for recent results.",
    func=ddg_search.run,
)

In [5]:
ddg_search_tool.run("Find an incident in machine learning pipeline.")

/home/mahsa/anaconda3/lib/python3.9/site-packages/duckduckgo_search/compat.py:20: UserWarning: ddg is deprecated. Use DDGS().text() generator
  warnings.warn("ddg is deprecated. Use DDGS().text() generator")
/home/mahsa/anaconda3/lib/python3.9/site-packages/duckduckgo_search/compat.py:22: UserWarning: parameter time is deprecated, use parameter timelimit
  warnings.warn("parameter time is deprecated, use parameter timelimit")
/home/mahsa/anaconda3/lib/python3.9/site-packages/duckduckgo_search/compat.py:24: UserWarning: parameter page is deprecated, use DDGS().text() generator
  warnings.warn("parameter page is deprecated, use DDGS().text() generator")
/home/mahsa/anaconda3/lib/python3.9/site-packages/duckduckgo_search/compat.py:26: UserWarning: parameter max_results is deprecated, use DDGS().text()
  warnings.warn("parameter max_results is deprecated, use DDGS().text()")


"05/29/2023 3 contributors Feedback In this article After submitting a pipeline, you'll see a link to the pipeline job in your Azure Machine Learning workspace. The link lands you in the pipeline job page in Azure Machine Learning studio, in which you can check result and debug your pipeline job. In this article, you learn how to troubleshoot when you get errors running a machine learning pipeline in the Azure Machine Learning SDK and Azure Machine Learning designer. Troubleshooting tips The following table contains common problems during pipeline development, with potential solutions. Authentication errors Published in. Acing AI. ·. 6 min read. ·. Apr 30, 2020. Incident management starts with realizing that there is an active incident with one of the ML application systems. Critical areas for ML ... 06/01/2023 6 contributors Feedback In this article APPLIES TO:Python SDK azureml v1 In this article, you learn how to create and run machine learning pipelinesby using the Azure Machine Le

## Da (Assistant)

In [6]:
da_llm = ChatOpenAI(temperature=0.0)

# Chain1 - get the incident

template_incident = """You are an incident solver. You are {personality}. Given an incident,\n
 it is your job to suggest an appropriate solution for it. You have {knowledge_base}.

Incident:
{incident}
Suggest solution:""".format(personality=teammates["Da (Assistant)"]['personality'], knowledge_base=teammates["Da (Assistant)"]['knowledge_base'], incident="{incident}")
prompt_template_incident = PromptTemplate(input_variables=["incident"], template=template_incident)
chain_one = LLMChain(llm=da_llm, prompt=prompt_template_incident) 

# Chain2 - you can search the internet for the solution
template_internet = """You are an incident solover that can use the internet to find the appropriate solution\n
 for the problem. You are {personality}. You have {knowledge_base}.

Incident:
{incident}
Suggest solution:""".format(personality=teammates["Da (Assistant)"]['personality'], knowledge_base=teammates["Da (Assistant)"]['knowledge_base'], incident="{incident}")
prompt_template_internet = PromptTemplate(input_variables=["incident"], template=template_internet)
chain_two = LLMChain(llm=da_llm, prompt=prompt_template_internet) 
da_overall_chain = SimpleSequentialChain(
                  chains=[chain_one, chain_two],
                  verbose=True)

In [7]:
# question = "An incident in the machine learning pipeline has happened. What can we do?"
question = incidents[1]["description"] + "What can we do?"
da_result = da_overall_chain.run(question)



> Entering new SimpleSequentialChain chain...
1. Identify the issue: Start by investigating the specific problem with the recent commit. Look for any error messages, logs, or any other indicators that can help pinpoint the root cause of the issue.

2. Rollback the commit: If the issue is critical and impacting the production environment, consider rolling back the recent commit to revert the changes and restore the system to its previous stable state. This can be done by reverting the commit or restoring a previous version of the code from a backup.

3. Analyze the code changes: Once the immediate issue is resolved, thoroughly review the code changes made in the recent commit. Look for any potential bugs, compatibility issues, or unintended consequences that might have caused the problem. Collaborate with the development team to understand the changes and identify any necessary fixes.

4. Test and validate: Before deploying any fixes or changes to the production environment, thoroughl

In [8]:
# print(da_result)

In [9]:
try:
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(da_result)

    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]

    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0)
    #chain = load_summarize_chain(llm, chain_type="map_reduce")
    chain = load_summarize_chain(llm, chain_type="stuff")
    da_summary = chain.run(docs)

    # Display summary
    print(color.BOLD + "Solution to the incident: " + color.END , da_summary)
except Exception as e:
    print("An error occurred: {}".format(e))

Solution to the incident:   This suggested solution outlines 8 steps to address an incident with a recent commit, including identifying the issue, rolling back the commit, analyzing the code changes, testing and validating, communicating with stakeholders, implementing the fix, monitoring and following up, and learning from the incident. The solution should be adapted to the specific incident and any additional steps may be necessary.


## Tyne (CTO)

In [10]:
tyne_llm = ChatOpenAI(temperature=0.0)

# Chain1 - get the incident

template_incident = """You are an incident solver. You are {personality}. Given an incident,\n
 it is your job to suggest an appropriate solution for it. You have {knowledge_base}.

Incident:
{incident}
Suggest solution:""".format(personality=teammates["Tyne (CTO)"]['personality'], knowledge_base=teammates["Tyne (CTO)"]['knowledge_base'], incident="{incident}")
prompt_template_incident = PromptTemplate(input_variables=["incident"], template=template_incident)
chain_one = LLMChain(llm=tyne_llm, prompt=prompt_template_incident) 

# Chain2 - you can search the internet for the solution
template_internet = """You are an incident solover that can use the internet to find the appropriate solution\n
 for the problem. You are {personality}. You have {knowledge_base}.

Incident:
{incident}
Suggest solution:""".format(personality=teammates["Tyne (CTO)"]['personality'], knowledge_base=teammates["Tyne (CTO)"]['knowledge_base'], incident="{incident}")
prompt_template_internet = PromptTemplate(input_variables=["incident"], template=template_internet)
chain_two = LLMChain(llm=tyne_llm, prompt=prompt_template_internet) 
tyne_overall_chain = SimpleSequentialChain(
                  chains=[chain_one, chain_two],
                  verbose=True)

In [11]:
question = incidents[1]["description"] + "What can we do?"
tyne_result = tyne_overall_chain.run(question)



> Entering new SimpleSequentialChain chain...
1. Identify the root cause: Start by investigating the recent commit that caused the issue. Analyze the code changes, dependencies, and any related logs or error messages to understand what went wrong.

2. Rollback the commit: If the issue is critical and impacting the production environment, consider rolling back the recent commit to restore the system to a stable state. This can be done by reverting the code changes and deploying the previous version that was working correctly.

3. Debug and fix the issue: If rolling back the commit is not feasible or doesn't resolve the problem, focus on debugging and fixing the issue. Use debugging tools, log analysis, and any available error information to identify the specific problem in the code. Once identified, make the necessary code changes to resolve the issue.

4. Test and validate the fix: After making the code changes, thoroughly test the fix in a non-production environment to ensure it res

In [12]:
try:
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(tyne_result)

    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]

    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0)
    #chain = load_summarize_chain(llm, chain_type="map_reduce")
    chain = load_summarize_chain(llm, chain_type="stuff")
    tyne_summary = chain.run(docs)

    # Display summary
    print(color.BOLD + "Solution to the incident: " + color.END , tyne_summary)
except Exception as e:
    print("An error occurred: {}".format(e))

Solution to the incident:   This suggested solution outlines a seven-step process for resolving an incident and preventing similar issues in the future. The steps involve identifying the root cause, rolling back the commit, debugging and fixing the issue, testing and validating the fix, deploying the fix to production, monitoring and verifying the fix, and learning from the incident. The solution also emphasizes the importance of leveraging analytical and technical skills, such as machine learning and data pipelines, to identify patterns and anomalies in the incident data.


## Ben (CEO)

In [13]:
ben_llm = ChatOpenAI(temperature=0.0)

# Chain1 - get the incident

template_incident = """You are an incident solver. You are {personality}. Given an incident,\n
 it is your job to suggest an appropriate solution for it. You have {knowledge_base}.

Incident:
{incident}
Suggest solution:""".format(personality=teammates["Ben (CEO)"]['personality'], knowledge_base=teammates["Ben (CEO)"]['knowledge_base'], incident="{incident}")
prompt_template_incident = PromptTemplate(input_variables=["incident"], template=template_incident)
chain_one = LLMChain(llm=ben_llm, prompt=prompt_template_incident) 

# Chain2 - you can search the internet for the solution
template_internet = """You are an incident solover that can use the internet to find the appropriate solution\n
 for the problem. You are {personality}. You have {knowledge_base}.

Incident:
{incident}
Suggest solution:""".format(personality=teammates["Ben (CEO)"]['personality'], knowledge_base=teammates["Ben (CEO)"]['knowledge_base'], incident="{incident}")
prompt_template_internet = PromptTemplate(input_variables=["incident"], template=template_internet)
chain_two = LLMChain(llm=ben_llm, prompt=prompt_template_internet) 
ben_overall_chain = SimpleSequentialChain(
                  chains=[chain_one, chain_two],
                  verbose=True)

In [14]:
question = incidents[1]["description"] + "What can we do?"
ben_result = ben_overall_chain.run(question)



> Entering new SimpleSequentialChain chain...
Solution: 

1. Identify the root cause: The first step is to investigate and identify the root cause of the issue with the recent commit. This can be done by reviewing the code changes, analyzing the logs, and involving the development team responsible for the commit.

2. Rollback the commit: If the issue is critical and impacting the production environment, it is advisable to rollback the recent commit to restore the system to its previous stable state. This can be done by reverting the code changes and deploying the previous version of the software.

3. Communicate with stakeholders: It is important to communicate the incident and the actions being taken to relevant stakeholders, including the development team, project managers, and any other affected parties. This will ensure transparency and keep everyone informed about the situation.

4. Conduct a post-mortem analysis: After resolving the immediate issue, it is crucial to conduct a t

In [15]:
try:
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(ben_result)

    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]

    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0)
    #chain = load_summarize_chain(llm, chain_type="map_reduce")
    chain = load_summarize_chain(llm, chain_type="stuff")
    ben_summary = chain.run(docs)

    # Display summary
    print(color.BOLD + "Solution to the incident: " + color.END , ben_summary)
except Exception as e:
    print("An error occurred: {}".format(e))

Solution to the incident:   In the event of an incident, the suggested solution is to identify the root cause, rollback the commit, communicate with stakeholders, conduct a post-mortem analysis, implement preventive measures, and monitor and review. It is important to approach the situation with a friendly and supportive attitude, emphasizing the importance of learning from mistakes and working together to prevent future incidents.


## Mix all the results

In [17]:
try:
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(da_summary+tyne_summary+ben_summary)

    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]

    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0)
    #chain = load_summarize_chain(llm, chain_type="map_reduce")
    chain = load_summarize_chain(llm, chain_type="stuff")
    overall_summary = chain.run(docs)

    # Display summary
    print(color.BOLD + "Solution to the incident: " + color.END , overall_summary)
except Exception as e:
    print("An error occurred: {}".format(e))

Solution to the incident:   This suggested solution outlines a seven-step process for resolving an incident and preventing similar issues in the future. The steps involve identifying the root cause, rolling back the commit, debugging and fixing the issue, testing and validating the fix, deploying the fix to production, monitoring and verifying the fix, and learning from the incident. Leveraging analytical and technical skills, such as machine learning and data pipelines, is also emphasized. The suggested solution should be adapted to the specific incident and any additional steps may be necessary.
